# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 29 2022 12:51PM,240756,10,Enova-10221,Emerginnova,Released
1,Jun 29 2022 12:50PM,240755,10,Enova-10222,Emerginnova,Released
2,Jun 29 2022 12:36PM,240753,10,8108681,"ACI Healthcare USA, Inc.",Released
3,Jun 29 2022 12:36PM,240753,10,8069286-BO,"ACI Healthcare USA, Inc.",Released
4,Jun 29 2022 12:36PM,240753,10,8108675-BO,"ACI Healthcare USA, Inc.",Released
5,Jun 29 2022 12:36PM,240753,10,8069292-BO1,"ACI Healthcare USA, Inc.",Released
6,Jun 29 2022 12:36PM,240753,10,8108679-BO,"ACI Healthcare USA, Inc.",Released
7,Jun 29 2022 12:36PM,240753,10,7994551-BO,"ACI Healthcare USA, Inc.",Released
8,Jun 29 2022 12:36PM,240750,20,8204800,"ACI Healthcare USA, Inc.",Released
9,Jun 29 2022 12:36PM,240750,20,8199770-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,240751,Released,1
37,240752,Released,11
38,240753,Released,6
39,240755,Released,1
40,240756,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240751,NaN,NaN,1.0
240752,NaN,NaN,11.0
240753,NaN,NaN,6.0
240755,NaN,NaN,1.0
240756,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240643,1.0,16.0,0.0
240648,0.0,5.0,8.0
240658,0.0,0.0,1.0
240662,0.0,0.0,1.0
240665,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240643,1,16,0
240648,0,5,8
240658,0,0,1
240662,0,0,1
240665,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240643,1,16,0
1,240648,0,5,8
2,240658,0,0,1
3,240662,0,0,1
4,240665,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240643,1,16,
1,240648,,5,8
2,240658,,,1
3,240662,,,1
4,240665,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 29 2022 12:51PM,240756,10,Emerginnova
1,Jun 29 2022 12:50PM,240755,10,Emerginnova
2,Jun 29 2022 12:36PM,240753,10,"ACI Healthcare USA, Inc."
8,Jun 29 2022 12:36PM,240750,20,"ACI Healthcare USA, Inc."
28,Jun 29 2022 12:36PM,240750,20,Else Nutrition
34,Jun 29 2022 12:32PM,240752,10,ISDIN Corporation
45,Jun 29 2022 12:27PM,240751,10,"Senseonics, Incorporated"
46,Jun 29 2022 12:25PM,240748,18,"ClearSpec, LLC"
47,Jun 29 2022 12:07PM,240747,20,"ACI Healthcare USA, Inc."
55,Jun 29 2022 12:06PM,240746,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 29 2022 12:51PM,240756,10,Emerginnova,,,1
1,Jun 29 2022 12:50PM,240755,10,Emerginnova,,,1
2,Jun 29 2022 12:36PM,240753,10,"ACI Healthcare USA, Inc.",,,6
3,Jun 29 2022 12:36PM,240750,20,"ACI Healthcare USA, Inc.",,,26
4,Jun 29 2022 12:36PM,240750,20,Else Nutrition,,,26
5,Jun 29 2022 12:32PM,240752,10,ISDIN Corporation,,,11
6,Jun 29 2022 12:27PM,240751,10,"Senseonics, Incorporated",,,1
7,Jun 29 2022 12:25PM,240748,18,"ClearSpec, LLC",,,1
8,Jun 29 2022 12:07PM,240747,20,"ACI Healthcare USA, Inc.",,,8
9,Jun 29 2022 12:06PM,240746,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 12:51PM,240756,10,Emerginnova,1,,
1,Jun 29 2022 12:50PM,240755,10,Emerginnova,1,,
2,Jun 29 2022 12:36PM,240753,10,"ACI Healthcare USA, Inc.",6,,
3,Jun 29 2022 12:36PM,240750,20,"ACI Healthcare USA, Inc.",26,,
4,Jun 29 2022 12:36PM,240750,20,Else Nutrition,26,,
5,Jun 29 2022 12:32PM,240752,10,ISDIN Corporation,11,,
6,Jun 29 2022 12:27PM,240751,10,"Senseonics, Incorporated",1,,
7,Jun 29 2022 12:25PM,240748,18,"ClearSpec, LLC",1,,
8,Jun 29 2022 12:07PM,240747,20,"ACI Healthcare USA, Inc.",8,,
9,Jun 29 2022 12:06PM,240746,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 12:51PM,240756,10,Emerginnova,1,,
1,Jun 29 2022 12:50PM,240755,10,Emerginnova,1,,
2,Jun 29 2022 12:36PM,240753,10,"ACI Healthcare USA, Inc.",6,,
3,Jun 29 2022 12:36PM,240750,20,"ACI Healthcare USA, Inc.",26,,
4,Jun 29 2022 12:36PM,240750,20,Else Nutrition,26,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 12:51PM,240756,10,Emerginnova,1.0,NaN,NaN
1,Jun 29 2022 12:50PM,240755,10,Emerginnova,1.0,NaN,NaN
2,Jun 29 2022 12:36PM,240753,10,"ACI Healthcare USA, Inc.",6.0,NaN,NaN
3,Jun 29 2022 12:36PM,240750,20,"ACI Healthcare USA, Inc.",26.0,NaN,NaN
4,Jun 29 2022 12:36PM,240750,20,Else Nutrition,26.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 12:51PM,240756,10,Emerginnova,1.0,0.0,0.0
1,Jun 29 2022 12:50PM,240755,10,Emerginnova,1.0,0.0,0.0
2,Jun 29 2022 12:36PM,240753,10,"ACI Healthcare USA, Inc.",6.0,0.0,0.0
3,Jun 29 2022 12:36PM,240750,20,"ACI Healthcare USA, Inc.",26.0,0.0,0.0
4,Jun 29 2022 12:36PM,240750,20,Else Nutrition,26.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3610888,60.0,23.0,0.0
12,481389,1.0,2.0,0.0
15,240648,8.0,5.0,0.0
18,722068,3.0,0.0,0.0
19,962817,28.0,11.0,0.0
20,1203635,78.0,19.0,1.0
21,240679,1.0,0.0,0.0
22,722150,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3610888,60.0,23.0,0.0
1,12,481389,1.0,2.0,0.0
2,15,240648,8.0,5.0,0.0
3,18,722068,3.0,0.0,0.0
4,19,962817,28.0,11.0,0.0
5,20,1203635,78.0,19.0,1.0
6,21,240679,1.0,0.0,0.0
7,22,722150,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,60.0,23.0,0.0
1,12,1.0,2.0,0.0
2,15,8.0,5.0,0.0
3,18,3.0,0.0,0.0
4,19,28.0,11.0,0.0
5,20,78.0,19.0,1.0
6,21,1.0,0.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,60.0
1,12,Released,1.0
2,15,Released,8.0
3,18,Released,3.0
4,19,Released,28.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,23.0,2.0,5.0,0.0,11.0,19.0,0.0,0.0
Released,60.0,1.0,8.0,3.0,28.0,78.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,23.0,2.0,5.0,0.0,11.0,19.0,0.0,0.0
2,Released,60.0,1.0,8.0,3.0,28.0,78.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,23.0,2.0,5.0,0.0,11.0,19.0,0.0,0.0
2,Released,60.0,1.0,8.0,3.0,28.0,78.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()